# Homework 2 - Recurrent Neural Networks


In this part of the homework we are going to work with Recurrent Neural Networks, in particular GRU. One of the greatest things that Recurrent Neural Networks can do when working with sequences is retaining data from several timesteps in the past. We are going to explore that property by constructing an 'echo' Recurrent Neural Network.

The goal here is to make a model that given a sequence of letters or digits will output that same sequence, but with a certain delay. Let's say the input is a string 'abacaba', we want the model to not output anything for 3 steps (delay length), and then output the original string step by step, except the last 3 characters. So, target output is then 'XXXabac', where 'X' is empty output.

This is similar to [this notebook](https://github.com/Atcold/pytorch-Deep-Learning/blob/master/09-echo_data.ipynb) (which you should refer to when doing this assignment), except we're working not with a binary string, but with a sequence of integers between 0 and some N. In our case N is 26, which is the number of letters in the alphabet.


## Dataset

Let's implement the dataset. In our case, the data is basically infinite, as we can always generate more examples on the fly, so there's no need to load it from disk.


In [39]:
import random
import string

import torch
import torch.nn.functional as F
from torch import nn
import torch.optim as optim
from tqdm.notebook import tqdm



# Max value of the generated integer. 26 is chosen because it's
# the number of letters in English alphabet.
N = 26


def idx_to_onehot(x, k=N+1):
    """ Converts the generated integers to one-hot vectors """
    ones = torch.sparse.torch.eye(k)
    shape = x.shape
    res = ones.index_select(0, x.view(-1).type(torch.int64))
    return res.view(*shape, res.shape[-1])


class EchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, delay=4, seq_length=15, size=1000):
        self.delay = delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        """ Iterable dataset doesn't have to implement __getitem__.
            Instead, we only need to implement __iter__ to return
            an iterator (or generator).
        """
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1))
                               for i in range(self.seq_length)], dtype=torch.int64)
            result = torch.cat(
                (torch.zeros(self.delay), seq[:self.seq_length - self.delay])).type(torch.int64)
            yield seq, result


DELAY = 4
DATASET_SIZE = 1000#200000
ds = EchoDataset(delay=DELAY, size=DATASET_SIZE)


## Model

Now, we want to implement the model. For our purposes, we want to use GRU. The architecture consists of GRU and a decoder. Decoder is responsible for decoding the GRU hidden state to yield a predicting for the next output. The parts you are responsible for filling with your code are marked with `TODO`.


In [40]:
input_size = N + 1
sequence_len = N + 1
num_classes = sequence_len
num_layers = 2 #stck em up
hidden_size = 64
import numpy as np

# making sure we are using GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)


class GRUMemory(torch.nn.Module):

    def __init__(self, input_size, hidden_size, num_layers, num_classes):
        super().__init__()
        # TODO: initialize your submodules

        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first = True)
        # x is [batch x seq x input]
        self.fc = nn.Linear(hidden_size, num_classes)
        self.relu = nn.ReLU()

    def forward(self, x):
        # inputs: x - input tensor of shape (batch_size, seq_length, N+1)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)
        # TODO implement forward pass

        # Hidden state init for first input to 0; forward propagation by passing in the input and hidden state into the model
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size)
        out, _ = self.gru(x, h0) #dont need hidden state
        # out: tensor of shape (batch_size, seq_length, hidden_size)
        # out (z, N+1, hidden size)

        # Decode the hidden state of the last time step
        out = out[:, -1, :]

        out = self.fc(self.relu(out))

        return out

    @torch.no_grad()
    def test_run(self, s):
        # This function accepts one string s containing lowercase characters a-z.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # Use ASCII tricks: https://www.asciitable.com/ where a is 97 to z being 122 and a space is 32
        alpha_to_num = torch.from_numpy(np.array([ord(i)-96 if ord(i) >= 97 else ord(i) - 32 for i in list(s)]))
        dis_fire = idx_to_onehot(alpha_to_num).unsqueeze(0).to(device)

        logits = self.forward(dis_fire)
        predict = torch.argmax(logits, dim = 2)
        return [chr(i + 96) if i >= 1 else chr(i+32) for i in predict[0,:]]

model = GRUMemory(input_size, hidden_size, num_layers, num_classes).to(device)


device: cuda


## Training

Below you need to implement the training of the model. We give you more freedom as for the implementation. The two limitations are that it has to execute within 10 minutes, and that error rate should be below 1%.


In [41]:
def test_model(model, sequence_length=15):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase)
                    for i in range(random.randint(15, 25))])
        result = model.test_run(s)
        for c1, c2 in zip(s[:-DELAY], result[DELAY:]):
            correct += int(c1 == c2)
        total += len(s) - DELAY

    return correct / total


In [42]:
def train(model, optimizer, criterion, device, batch):

	model.train()
	data, target = idx_to_onehot(batch[0], k=N+1).to(device), batch[1].to(device)
	output = model(data)
	count_batch_correct = 0

	'''Prof Note:
		For loss functions like CrossEntropyLoss,
        # the second argument is actually expected to be a tensor of class indices rather than
        # one-hot encoded class labels. One approach is to take advantage of the one-hot encoding
        # of the target and call argmax along its second dimension to create a tensor of shape
        # (batch_size) containing the index of the class label that was hot for each sequence.'''
	predict = torch.argmax(output, dim = 2)
	target = torch.argmax(target, dim = 2)
	loss = criterion(output, target)

	optimizer.zero_grad()
	loss.backward()
	optimizer.step()

	matches = target == predict
	count_batch_correct, tot_batch = np.sum(matches), loss.item()

	return tot_batch, count_batch_correct

In [43]:
def eval(model, optimizer, criterion, device, batch):
	
	model.eval()
	data, target = idx_to_onehot(batch[0], k=N+1).to(device), batch[1].to(device)
	output = model(data)
	count_batch_correct = 0

	'''Prof Note:
		For loss functions like CrossEntropyLoss,
        # the second argument is actually expected to be a tensor of class indices rather than
        # one-hot encoded class labels. One approach is to take advantage of the one-hot encoding
        # of the target and call argmax along its second dimension to create a tensor of shape
        # (batch_size) containing the index of the class label that was hot for each sequence.'''
	predict = torch.argmax(output, dim = 2)
	target = torch.argmax(target, dim = 2)
	loss = criterion(output, target)

	optimizer.zero_grad()
	loss.backward()
	optimizer.step()

	matches = target == predict
	count_batch_correct, tot_batch = np.sum(matches), loss.item()

	return tot_batch, count_batch_correct

In [44]:
import time

BATCH_SIZE = 64
N_EPOCHS = 100

# TODO
dataset_size = len(ds)
test_split = 0.4
test_size = int(test_split * dataset_size)
train_size = dataset_size - test_size

train_dataset, validation_dataset = torch.utils.data.random_split(ds, [train_size, test_size])
train_dataloader = torch.utils.data.DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
validation_dataloader = torch.utils.data.DataLoader(
    validation_dataset, batch_size=BATCH_SIZE, num_workers=4)

criterion = nn.CrossEntropyLoss()

if torch.cuda.is_available():
    model = model.cuda()
    criterion = criterion.cuda()
    device = torch.device('cuda:0')
    print(f'device: {device}')
else:
    device = torch.device('cpu')

model.to(device) 
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=2.75e-4)


start_time = time.time()
train_losses = []
train_accuracies = []
validation_losses = []
validation_accuracies = []

pbar = tqdm(range(N_EPOCHS))

for i in pbar:
    total_train_correct = 0.0
    total_validation_correct = 0.0
    total_train = 0
    total_val = 0

    model.train()

    for batch in tqdm(train_dataloader, leave=False):
        tot_batch, count_batch_correct = train(model=model,optimizer=optimizer,criterion=criterion,device=device,batch=batch)
        total_train += tot_batch
        total_train_correct += count_batch_correct

    with torch.no_grad():
        for batch in validation_dataloader:
            tot_batch, count_batch_correct = eval(model=model,optimizer=optimizer,criterion=criterion,device=device,batch=batch)
            total_val += tot_batch
            total_validation_correct += count_batch_correct

    mean_train_loss = total_train / len(train_dataset)
    train_accuracy = total_train_correct / len(train_dataset)

    mean_validation_loss = total_val / len(validation_dataset)
    validation_accuracy = total_validation_correct / len(validation_dataset)

    train_losses.append(mean_train_loss)
    validation_losses.append(mean_validation_loss)

    train_accuracies.append(train_accuracy)
    validation_accuracies.append(validation_accuracy)

    pbar.set_postfix({'train_loss': mean_train_loss, 'validation_loss': mean_validation_loss,
                     'train_accuracy': train_accuracy, 'validation_accuracy': validation_accuracy})

end_time = time.time()
duration = end_time - start_time
accuracy = test_model(model)
assert duration < 600, 'execution took f{duration:.2f} seconds, which longer than 10 mins'
assert accuracy > 0.99, f'accuracy is too low, got {accuracy}, need 0.99'
print('tests passed')


device: cuda:0


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

RuntimeError: DataLoader worker (pid(s) 7636, 17800, 29012, 39816) exited unexpectedly

## Variable delay model

Now, to make this more complicated, we want to have variable delay. So, now, the goal is to transform a sequence of pairs (character, delay) into a character sequence with given delay. Delay is constant within one sequence.


### Dataset

As before, we first implement the dataset:


In [ ]:
class VariableDelayEchoDataset(torch.utils.data.IterableDataset):

    def __init__(self, max_delay=8, seq_length=20, size=1000):
        self.max_delay = max_delay
        self.seq_length = seq_length
        self.size = size

    def __len__(self):
        return self.size

    def __iter__(self):
        for _ in range(self.size):
            seq = torch.tensor([random.choice(range(1, N + 1))
                               for i in range(self.seq_length)], dtype=torch.int64)
            delay = random.randint(0, self.max_delay)
            result = torch.cat(
                (torch.zeros(delay), seq[:self.seq_length - delay])).type(torch.int64)
            yield seq, delay, result


### Model

And the model.


In [ ]:
class VariableDelayGRUMemory(torch.nn.Module):

    def __init__(self, hidden_size, max_delay):
        super().__init__()
        # TODO

    def forward(self, x, delays):
        # inputs:
        # x - tensor of shape (batch size, seq length, N + 1)
        # delays - tensor of shape (batch size)
        # returns:
        # logits (scores for softmax) of shape (batch size, seq_length, N + 1)

        # TODO
        pass

    @torch.no_grad()
    def test_run(self, s, delay):
        # This function accepts one string s containing lowercase characters a-z,
        # and a delay - the desired output delay.
        # You need to map those characters to one-hot encodings,
        # then get the result from your network, and then convert the output
        # back to a string of the same length, with 0 mapped to ' ',
        # and 1-26 mapped to a-z.

        # TODO
        pass


### Train

As before, you're free to do what you want, as long as training finishes within 10 minutes and accuracy is above 0.99 for delays between 0 and 8.


In [ ]:
def test_variable_delay_model(model, seq_length=20):
    """
    This is the test function that runs 100 different strings through your model,
    and checks the error rate.
    """
    total = 0
    correct = 0
    for i in range(500):
        s = ''.join([random.choice(string.ascii_lowercase)
                    for i in range(seq_length)])
        d = random.randint(0, model.max_delay)
        result = model.test_run(s, d)
        if d > 0:
            z = zip(s[:-d], result[d:])
        else:
            z = zip(s, result)
        for c1, c2 in z:
            correct += int(c1 == c2)
        total += len(s) - d

    return correct / total


In [ ]:
import time
start_time = time.time()

MAX_DELAY = 8
SEQ_LENGTH = 20

# TODO: implement model training here.
model = None


end_time = time.time()
assert end_time - start_time < 600, 'executing took longer than 10 mins'
assert test_variable_delay_model(model) > 0.99, 'accuracy is too low'
print('tests passed')
